# generate data for toy problem

In [16]:
import random
a = 1
b = 2
c = 3
X1 = []
X2 = []
X3 = []
for i in range(100):
    x1 = []
    x2 = []
    x3 = []
    for i in range(7):

        x1.append(random.uniform(-1,1))
        x2.append(random.uniform(-1,1))
        x3.append(random.uniform(-1,1))
    X1.append(x1)
    X2.append(x2)
    X3.append(x3)

Y = []
for i in range(100):
    for j in range(7):
        y1 = X1[i][j] * a
        y2 = X2[i][j] * b
        y3 = X3[i][j] * c
        y = y1 + y2 + y3
    Y.append(y)

import pandas as pd
import numpy as np
X1 = pd.DataFrame(X1)
X2 = pd.DataFrame(X2)
X3 = pd.DataFrame(X3)
Y = pd.DataFrame(Y)
result = pd.concat([X1, X2], axis=1)
result = pd.concat([result,X3],axis = 1)
result = pd.concat([result,Y],axis = 1)
result = result.values
result
np.savetxt('toy problem.csv', result, delimiter = ',')

# Nearest neighbour

In [19]:
import numpy as np
np.set_printoptions(suppress=True)

#import row data
row_data = np.loadtxt('toy problem.csv',delimiter = ',')
np.random.seed(1)
np.random.shuffle(row_data)
X = row_data[:,:21]
Y = row_data[:,21:]

# normalize_x
X = (X - np.mean(X)) / np.std(X)

#split data
split_size = [0.6,0.2,0.2]
index_train = int(X.shape[0] * split_size[0])
index_cv = index_train + int(X.shape[0] * split_size[1])
index_test = index_cv + int(X.shape[0] * split_size[2])
X_train = X[:index_train, :]
Y_train = Y[:index_train, :]
X_test = X[index_cv:index_test, :]
Y_test = Y[index_cv:index_test, :]

best_k = -100
best_rmse = 10000000000000000000000
for k in range(1,10):
    size1 = Y_test.shape
    X_predicted = np.zeros(size1)
    size2 = X_predicted.shape[0]
    for i in range(size2):
        distance = np.linalg.norm(X_train - X_test[i], axis=1)
        index = np.argpartition(distance, k)[:k]
        weights = 1 / distance[index]**2
        X_predicted[i][0] = np.sum(np.squeeze(Y_train[index]) \
                                   * np.squeeze(weights))/ np.sum(weights)
    rmse = np.sqrt(np.mean(np.square(X_predicted - Y_test)))
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k 

print("Nearest neighbor:")
print("k:{}".format(best_k))
print("RMSE: {}.".format(rmse))

Nearest neighbor:
k:8
RMSE: 1.6086403219302157.


# Linear regression

In [20]:
n = X_train.shape[1]
w = np.random.normal(0, 0.5, size=(n, 1))
b = np.random.normal(0, 0.5)

iters = 10000
learning_rate = 0.1

costed = 100000000000000000000
new_learning_rate = learning_rate

for i in range(iters):
    prediction = np.dot(X_train, w) + b
    cost = np.sum((prediction - Y_train)**2) / (2 * Y_train.shape[0])
    m = prediction.shape[0]
    grad_w = np.dot(X_train.T, (prediction - Y_train)) / m
    grad_b = np.sum((prediction - Y_train)) / m
    w = w - learning_rate * grad_w
    b = b - learning_rate * grad_b      
    if cost >= costed:
        new_learning_rate = new_learning_rate * 0.5
        continue
        
    costed = cost

prediction = np.dot(X_test, w) + b
rmse = np.sqrt(np.mean(np.square(prediction - Y_test)))    

print("Linear regression :")
print("RMSE: {}.".format(rmse))

Linear regression :
RMSE: 3.704657195003952e-15.


# Regression forest 

# Gaussian process